# Main Idea

1. only look at base since contributes_to can only occur between bases
2. assign score of 1 where there is a relationship and 0 to the reverse of it
3. train a classifier model

In [1]:
import srsly
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk

In [2]:
rel_label = ["Contributes_To"]

In [3]:
def prep_data(rel_labels):
    file_name_answers = "checkin_answers"
    file_path_answers = "/Users/ash/Desktop/" + file_name_answers + ".jsonl"
    data = srsly.read_jsonl(file_path_answers)
    final_sent = []

    for entry in data:
        if "text" in entry:
            text = entry["text"]
        label_arr = []
        label_dict = {}
        if entry['answer'] == "accept":
            if entry['_session_id'] == "main_3_per_cluster-Kameron":
                for relation in entry['relations']:
                    if ("label" in relation) and ("head_span" in relation) and ("child_span" in relation):
                        if relation["label"] in rel_labels:
                            head_span_start = relation["head_span"]["start"]
                            head_span_end = relation["head_span"]["end"]
                            child_span_start = relation["child_span"]["start"]
                            child_span_end = relation["child_span"]["end"]
                            head_word = text[head_span_start:head_span_end]
                            child_word = text[child_span_start:child_span_end]
                            tmp_dict = {"head": head_word, "child": child_word, "label": relation["label"]}
                            label_arr.append(tmp_dict)
                if len(label_arr) > 0:
                    label_tup = (text, {"entities": label_arr})
                    final_sent.append(label_tup)
                else:
                    pass
                    #print(text)
    return final_sent
                    
final_sent = prep_data(rel_label)
print(final_sent)

[('These risks are crop insurance, health care, wildfire suppression, hurricane-related disaster relief, and federal facility flood risk, all of which are anticipated to cost billions of dollars more by the end of the century due to the impacts of climate change.23 IMPLIED_BASE IMPLIED_BASE', {'entities': [{'head': 'climate change.23', 'child': 'insurance', 'label': 'Contributes_To'}, {'head': 'climate change.23', 'child': 'health care', 'label': 'Contributes_To'}, {'head': 'climate change.23', 'child': 'wildfire suppression', 'label': 'Contributes_To'}, {'head': 'climate change.23', 'child': 'disaster relief', 'label': 'Contributes_To'}, {'head': 'climate change.23', 'child': 'flood', 'label': 'Contributes_To'}]}), ('The IPCC reported that Alexandria’s beaches would be submerged even with a 0.5-metre sea-level rise, while 8 million people would be displaced by flooding in Alexandria and the Nile Delta if no protective measures are taken. IMPLIED_BASE IMPLIED_BASE', {'entities': [{'hea

In [4]:
def getPOS(sent):
    pos = {}
    doc = nlp(sent)
    for token in doc:
        pos[token.text] = token.pos_
    return pos

In [5]:
csv = []
#features = ["head", "child", "POS_head", "POS_child", "prediction"]
features = ["head", "child", "prediction"]
csv.append(features)

for x in final_sent:
    #pos = getPOS(x[0])
    for y in x[1]['entities']:
        #tmp = [y["head"], y["child"], pos[y["head"]], pos[y["child"]], "1"]
        tmp = [y["head"], y["child"], "1"]
        csv.append(tmp)
        tmp = []
        
        #tmp = [y["child"], y["head"], pos[y["child"]], pos[y["head"]], "0"]
        tmp = [y["child"], y["head"], "0"]
        csv.append(tmp)
        tmp = []
print(csv)

[['head', 'child', 'prediction'], ['climate change.23', 'insurance', '1'], ['insurance', 'climate change.23', '0'], ['climate change.23', 'health care', '1'], ['health care', 'climate change.23', '0'], ['climate change.23', 'wildfire suppression', '1'], ['wildfire suppression', 'climate change.23', '0'], ['climate change.23', 'disaster relief', '1'], ['disaster relief', 'climate change.23', '0'], ['climate change.23', 'flood', '1'], ['flood', 'climate change.23', '0'], ['sea', 'beaches', '1'], ['beaches', 'sea', '0'], ['flooding', 'people', '1'], ['people', 'flooding', '0'], ['heat wave', 'ice sheet', '1'], ['ice sheet', 'heat wave', '0'], ['IMPLIED_BASE', 'heat wave', '1'], ['heat wave', 'IMPLIED_BASE', '0'], ['IMPLIED_BASE', 'rain', '1'], ['rain', 'IMPLIED_BASE', '0'], ['rain', 'flooding', '1'], ['flooding', 'rain', '0'], ['rain', 'nutrients', '1'], ['nutrients', 'rain', '0'], ['Hurricane', 'PTSD', '1'], ['PTSD', 'Hurricane', '0'], ['Hurricane', 'depression', '1'], ['depression', 'Hu

In [6]:
import pandas as pd

df = pd.DataFrame(i for i in csv)
df.drop(0, axis=0, inplace=True)
df.columns = csv[0]
df

,head,child,prediction
1,climate change.23,insurance,1
2,insurance,climate change.23,0
3,climate change.23,health care,1
4,health care,climate change.23,0
5,climate change.23,wildfire suppression,1
...,...,...,...
1640,island,oil,0
1641,air,urban heat island effect,1
1642,urban heat island effect,air,0
1643,pavement,air,1


# Logistic Regression

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

feature_cols = ["head", "child"]
X = df[feature_cols] # Features

# one-hot encodding for categorical variables
X = pd.get_dummies(data=X, drop_first=True)
y = df.prediction

X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, random_state=0)

In [8]:
from sklearn.linear_model import LogisticRegression
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

In [9]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[147,  56],
       [ 50, 158]])

In [28]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.7420924574209246


# SVM

In [37]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred_svm = clf.predict(X_test)

In [38]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred_svm)
cnf_matrix

array([[150,  53],
       [ 47, 161]])

In [39]:
print(accuracy_score(y_test, y_pred_svm))

0.7566909975669099
